In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder

from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import IsolationForest

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score

In [2]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [3]:
#Remove the ID column
df.info()
df.drop('id', axis=1, inplace=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
DT_bmi_pipe = Pipeline( steps=[ 
                               ('scale',StandardScaler()),
                               ('lr',DecisionTreeRegressor(random_state=0))
                              ])
X = df[['age','gender','bmi']].copy()
X.gender = X.gender.replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)

Missing = X[X.bmi.isna()]
X = X[~X.bmi.isna()]
Y = X.pop('bmi')
DT_bmi_pipe.fit(X,Y)
predicted_bmi = pd.Series(DT_bmi_pipe.predict(Missing[['age','gender']]),index=Missing.index)
df.loc[Missing.index,'bmi'] = predicted_bmi

In [5]:
##Label Encode Binary
residence_type = df['Residence_type'].unique()
ever_married = df['ever_married'].unique()
le = LabelEncoder()

le.fit(residence_type)
df['Residence_type'] = le.transform(df['Residence_type'])

le.fit(ever_married)
df['ever_married'] = le.transform(df['ever_married'])

In [6]:
df
#Label Encode -> One Hot Encode -> PCA -> Splitting -> Outlier Detection -> SMOTE X_train and y_train -> Bayes Training

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,1,Private,1,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,1,Self-employed,0,202.21,29.879487,never smoked,1
2,Male,80.0,0,1,1,Private,0,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,1,Private,1,171.23,34.400000,smokes,1
4,Female,79.0,1,0,1,Self-employed,0,174.12,24.000000,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,1,Private,1,83.75,28.476923,never smoked,0
5106,Female,81.0,0,0,1,Self-employed,1,125.20,40.000000,never smoked,0
5107,Female,35.0,0,0,1,Self-employed,0,82.99,30.600000,never smoked,0
5108,Male,51.0,0,0,1,Private,0,166.29,25.600000,formerly smoked,0


In [7]:
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 
       'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']

In [8]:
#One Hot Encode everything
encoder = ColumnTransformer(transformers=[('onehot', OneHotEncoder(), ['gender', 'work_type', 'smoking_status'])], remainder='passthrough')
X = np.array(encoder.fit_transform(X))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
#Standardize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
#Isolation Forest
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)

In [12]:
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]

In [13]:
#ROS
X_ros, y_ros = RandomOverSampler().fit_resample(X_train, y_train)

# ANN

In [14]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential  
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import regularizers

In [15]:
def ann_classifier(optimizer = 'adam'):
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
    model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
    tf.keras.layers.Dropout(0.6)
    model.add(layers.Dense(1, activation=tf.nn.sigmoid))
    model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [16]:
ann = KerasClassifier(build_fn = ann_classifier, batch_size = 32, epochs = 50)

# Before Tuning

In [17]:
prior_model = tf.keras.models.Sequential()
prior_model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
prior_model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
tf.keras.layers.Dropout(0.4)
prior_model.add(layers.Dense(1, activation=tf.nn.sigmoid))
prior_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
epochs = 100
prior_model.fit(X_ros, y_ros, epochs=epochs)

Epoch 1/100
221/221 [==============================] - 2s 725us/step - loss: 0.7569 - accuracy: 0.6669
Epoch 2/100
221/221 [==============================] - 0s 486us/step - loss: 0.6131 - accuracy: 0.7416
Epoch 3/100
221/221 [==============================] - 0s 500us/step - loss: 0.5513 - accuracy: 0.7707
Epoch 4/100
221/221 [==============================] - 0s 482us/step - loss: 0.5239 - accuracy: 0.7801
Epoch 5/100
221/221 [==============================] - 0s 477us/step - loss: 0.5067 - accuracy: 0.7836
Epoch 6/100
221/221 [==============================] - 0s 491us/step - loss: 0.4971 - accuracy: 0.7886
Epoch 7/100
221/221 [==============================] - 0s 491us/step - loss: 0.4902 - accuracy: 0.7876
Epoch 8/100
221/221 [==============================] - 0s 486us/step - loss: 0.4840 - accuracy: 0.7908
Epoch 9/100
221/221 [==============================] - 0s 486us/step - loss: 0.4783 - accuracy: 0.7890
Epoch 10/100
221/221 [==============================] - 0s 496us/step - l

221/221 [==============================] - 0s 582us/step - loss: 0.3636 - accuracy: 0.8688
Epoch 80/100
221/221 [==============================] - 0s 577us/step - loss: 0.3608 - accuracy: 0.8738
Epoch 81/100
221/221 [==============================] - 0s 555us/step - loss: 0.3605 - accuracy: 0.8712
Epoch 82/100
221/221 [==============================] - 0s 536us/step - loss: 0.3604 - accuracy: 0.8742
Epoch 83/100
221/221 [==============================] - 0s 536us/step - loss: 0.3609 - accuracy: 0.8718
Epoch 84/100
221/221 [==============================] - 0s 559us/step - loss: 0.3600 - accuracy: 0.8732
Epoch 85/100
221/221 [==============================] - 0s 536us/step - loss: 0.3589 - accuracy: 0.8723
Epoch 86/100
221/221 [==============================] - 0s 532us/step - loss: 0.3590 - accuracy: 0.8714
Epoch 87/100
221/221 [==============================] - 0s 550us/step - loss: 0.3574 - accuracy: 0.8723
Epoch 88/100
221/221 [==============================] - 0s 541us/step - loss:

In [19]:
y_pred=(prior_model.predict(X_test) > 0.5).astype("int32")

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)
roc = roc_auc_score(y_test, y_pred)
print('ROC score: %f' % roc)
print('----------------------------')
print(classification_report(y_test, y_pred))

Accuracy: 0.730920
Precision: 0.117647
Recall: 0.629630
F1 score: 0.198251
ROC score: 0.683100
----------------------------
              precision    recall  f1-score   support

           0       0.97      0.74      0.84       968
           1       0.12      0.63      0.20        54

    accuracy                           0.73      1022
   macro avg       0.55      0.68      0.52      1022
weighted avg       0.93      0.73      0.80      1022



# Tune ANN

In [20]:
parameters = {'batch_size': [25, 32],
             'epochs': [50, 100, 150],
             'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = ann, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)

grid_search.fit(X_ros, y_ros)

Epoch 1/150
283/283 [==============================] - 1s 471us/step - loss: 0.8061 - accuracy: 0.6326
Epoch 2/150
283/283 [==============================] - 0s 475us/step - loss: 0.6360 - accuracy: 0.7460
Epoch 3/150
283/283 [==============================] - 0s 472us/step - loss: 0.5565 - accuracy: 0.7765
Epoch 4/150
283/283 [==============================] - 0s 468us/step - loss: 0.5237 - accuracy: 0.7862
Epoch 5/150
283/283 [==============================] - 0s 504us/step - loss: 0.5079 - accuracy: 0.7917
Epoch 6/150
283/283 [==============================] - 0s 489us/step - loss: 0.4968 - accuracy: 0.7934
Epoch 7/150
283/283 [==============================] - 0s 475us/step - loss: 0.4887 - accuracy: 0.7956
Epoch 8/150
283/283 [==============================] - 0s 473us/step - loss: 0.4830 - accuracy: 0.7952
Epoch 9/150
283/283 [==============================] - 0s 474us/step - loss: 0.4776 - accuracy: 0.7985
Epoch 10/150
283/283 [==============================] - 0s 472us/step - l

283/283 [==============================] - 0s 475us/step - loss: 0.3981 - accuracy: 0.8384
Epoch 80/150
283/283 [==============================] - 0s 493us/step - loss: 0.3980 - accuracy: 0.8425
Epoch 81/150
283/283 [==============================] - 0s 475us/step - loss: 0.3966 - accuracy: 0.8394
Epoch 82/150
283/283 [==============================] - 0s 482us/step - loss: 0.3974 - accuracy: 0.8368
Epoch 83/150
283/283 [==============================] - 0s 461us/step - loss: 0.3961 - accuracy: 0.8428
Epoch 84/150
283/283 [==============================] - 0s 468us/step - loss: 0.3972 - accuracy: 0.8401
Epoch 85/150
283/283 [==============================] - 0s 458us/step - loss: 0.3944 - accuracy: 0.8429
Epoch 86/150
283/283 [==============================] - 0s 458us/step - loss: 0.3954 - accuracy: 0.8395
Epoch 87/150
283/283 [==============================] - 0s 482us/step - loss: 0.3948 - accuracy: 0.8438
Epoch 88/150
283/283 [==============================] - 0s 458us/step - loss:

GridSearchCV(cv=10,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000156B4D0EB50>,
             n_jobs=-1,
             param_grid={'batch_size': [25, 32], 'epochs': [50, 100, 150],
                         'optimizer': ['adam', 'rmsprop']},
             scoring='accuracy')

In [21]:
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 87.23 %
Best Parameters: {'batch_size': 25, 'epochs': 150, 'optimizer': 'adam'}


# Running ANN after Tuning

In [26]:
model = tf.keras.models.Sequential()
model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
tf.keras.layers.Dropout(0.4)
model.add(layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
epochs = 150
model.fit(X_ros, y_ros, batch_size= 25, epochs=epochs)

Epoch 1/150
283/283 [==============================] - 1s 468us/step - loss: 0.8029 - accuracy: 0.6475
Epoch 2/150
283/283 [==============================] - 0s 482us/step - loss: 0.6170 - accuracy: 0.7673
Epoch 3/150
283/283 [==============================] - 0s 472us/step - loss: 0.5473 - accuracy: 0.7843
Epoch 4/150
283/283 [==============================] - 0s 468us/step - loss: 0.5176 - accuracy: 0.7907
Epoch 5/150
283/283 [==============================] - 0s 465us/step - loss: 0.5009 - accuracy: 0.7965
Epoch 6/150
283/283 [==============================] - 0s 460us/step - loss: 0.4903 - accuracy: 0.8022
Epoch 7/150
283/283 [==============================] - 0s 468us/step - loss: 0.4813 - accuracy: 0.7978
Epoch 8/150
283/283 [==============================] - 0s 461us/step - loss: 0.4751 - accuracy: 0.7993
Epoch 9/150
283/283 [==============================] - 0s 460us/step - loss: 0.4690 - accuracy: 0.8085
Epoch 10/150
283/283 [==============================] - 0s 458us/step - l

283/283 [==============================] - 0s 461us/step - loss: 0.3558 - accuracy: 0.8753
Epoch 80/150
283/283 [==============================] - 0s 465us/step - loss: 0.3559 - accuracy: 0.8750
Epoch 81/150
283/283 [==============================] - 0s 468us/step - loss: 0.3561 - accuracy: 0.8750
Epoch 82/150
283/283 [==============================] - 0s 475us/step - loss: 0.3561 - accuracy: 0.8769
Epoch 83/150
283/283 [==============================] - 0s 461us/step - loss: 0.3536 - accuracy: 0.8789
Epoch 84/150
283/283 [==============================] - 0s 459us/step - loss: 0.3536 - accuracy: 0.8798
Epoch 85/150
283/283 [==============================] - 0s 450us/step - loss: 0.3529 - accuracy: 0.8801
Epoch 86/150
283/283 [==============================] - 0s 454us/step - loss: 0.3525 - accuracy: 0.8813
Epoch 87/150
283/283 [==============================] - 0s 459us/step - loss: 0.3523 - accuracy: 0.8774
Epoch 88/150
283/283 [==============================] - 0s 458us/step - loss:

In [24]:
test_loss, test_acc = model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 419us/step - loss: 0.5949 - accuracy: 0.7397


In [25]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)
roc = roc_auc_score(y_test, y_pred)
print('ROC score: %f' % roc)
print('----------------------------')
print(classification_report(y_test, y_pred))

Accuracy: 0.739726
Precision: 0.124113
Recall: 0.648148
F1 score: 0.208333
ROC score: 0.696491
----------------------------
              precision    recall  f1-score   support

           0       0.97      0.74      0.84       968
           1       0.12      0.65      0.21        54

    accuracy                           0.74      1022
   macro avg       0.55      0.70      0.53      1022
weighted avg       0.93      0.74      0.81      1022

